### Transportation-Networks Problem

In [29]:
from pyomo.environ import *

In [5]:
Demand = {
    'Lon': 125,
    'Ber': 175,
    'Maa': 225,
    'Ams': 250,
    'Utr': 225,
    'Hag': 200
}

Supply = {
    'Arn':600,
    'Gou':650
}

T = {
    ('Lon', 'Arn'): 1000,
    ('Lon', 'Gou'): 2.5,
    ('Ber', 'Arn'): 2.5,
    ('Ber', 'Gou'): 1000,
    ('Maa', 'Arn'): 1.6,
    ('Maa', 'Gou'): 2.0,
    ('Ams', 'Arn'): 1.4,
    ('Ams', 'Gou'): 1.0,
    ('Utr', 'Arn'): 0.8,
    ('Utr', 'Gou'): 1.0,
    ('Hag', 'Arn'): 1.4,
    ('Hag', 'Gou'): 0.8
}

In [80]:
# Step 0
model = ConcreteModel()
model.dual = Suffix(direction = Suffix.IMPORT)

# Step 1: Define index sets
CUS = list(Demand.keys())
SRC = list(Supply.keys())

# Step 2: Define the decision
model.x = Var(CUS, SRC, domain = NonNegativeReals)

@model.Objective(sense = minimize)
def custom(m):
    return sum(T[c,s]*model.x[c,s] for c in CUS for s in SRC)

@model.Constraint(SRC)
def src(m, s):
    return sum([model.x[c,s] for c in CUS]) <= Supply[s]

@model.Constraint(CUS)
def dmd(m, c):
    return sum([model.x[c,s] for s in SRC]) >= Demand[c]
results = SolverFactory('cbc').solve(model)
results.write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1705.0
  Upper bound: 1705.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 12
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of create

In [83]:
CUS

['Lon', 'Ber', 'Maa', 'Ams', 'Utr', 'Hag']

In [81]:
model.x.display()

x : Size=12, Index=x_index
    Key            : Lower : Value : Upper : Fixed : Stale : Domain
    ('Ams', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Ams', 'Gou') :     0 : 250.0 :  None : False : False : NonNegativeReals
    ('Ber', 'Arn') :     0 : 175.0 :  None : False : False : NonNegativeReals
    ('Ber', 'Gou') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Hag', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Hag', 'Gou') :     0 : 200.0 :  None : False : False : NonNegativeReals
    ('Lon', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Lon', 'Gou') :     0 : 125.0 :  None : False : False : NonNegativeReals
    ('Maa', 'Arn') :     0 : 225.0 :  None : False : False : NonNegativeReals
    ('Maa', 'Gou') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Utr', 'Arn') :     0 : 200.0 :  None : False : False : NonNegativeReals
    ('Utr', 'Gou') :     0 :  25.0 :  None : Fa

In [50]:
for c in CUS:
    for s in SRC:
        print(c, s, model.x[c,s]())

Lon Arn 0.0
Lon Gou 125.0
Ber Arn 175.0
Ber Gou 0.0
Maa Arn 225.0
Maa Gou 0.0
Ams Arn 0.0
Ams Gou 250.0
Utr Arn 200.0
Utr Gou 25.0
Hag Arn 0.0
Hag Gou 200.0


In [49]:
list(model.dmd.keys())

['Lon', 'Ber', 'Maa', 'Ams', 'Utr', 'Hag']

In [27]:
str = "   {0:7.2f} {1:7.2f} {2:7.2f} {3:7.2f}"

print("Constraint  value  lslack  uslack    dual")
for c in model.x:
    print(c, str.format(c(), c.lslack(), c.uslack(), model.dual[c]))
    

Constraint  value  lslack  uslack    dual
('Lon', 'Arn')
('Lon', 'Gou')
('Ber', 'Arn')
('Ber', 'Gou')
('Maa', 'Arn')
('Maa', 'Gou')
('Ams', 'Arn')
('Ams', 'Gou')
('Utr', 'Arn')
('Utr', 'Gou')
('Hag', 'Arn')
('Hag', 'Gou')


In [60]:
    for s in SRC:
        print(f"{s:10s}{Supply[s]:10.1f}{model.src[s]():10.1f}{model.dual[model.src[s]]:10.4f}")

Arn            600.0     600.0   -0.2000
Gou            650.0     600.0    0.0000


In [79]:
model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   2.0 :  None : False : False : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   6.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 3*x + 5*y

3 Constraint Declarations
    fab_1 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :   4.0 :   True
    fab_2 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :  2*y :  12.0 :   True
    fab_3 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : 3*x + 2*y :  18.0 :   True

1 Suffix Declarations
    dual : Direction=Suffix.IMPORT,

In [65]:
model.src[s].lslack()

inf

In [39]:
model = ConcreteModel()
# for access to dual solution for constraints
model.dual = Suffix(direction=Suffix.IMPORT)

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
# model.profit = Objective(
#     expr = 40*model.x + 30*model.y,
#     sense = maximize)

@model.Objective(sense = maximize)
def profit(m):
    return 40*model.x + 30*model.y

# declare constraints
#model.demand = Constraint(expr = model.x <= 40)
#model.laborA = Constraint(expr = model.x + model.y <= 80)
#model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

@model.Constraint()
def demand(m):
    return model.x <= 40

@model.Constraint()
def laborA(m):
    return model.x + model.y <= 80

@model.Constraint()
def laborB(m):
    return 2*model.x + model.y <= 100

# solve
SolverFactory('cbc').solve(model)

print("\nSolution")
print(f"x = {model.x()}")
print(f"y = {model.y()}")

print("\nSensitivity Analysis")
print(f"y_demand = {-model.dual[model.demand]}")
print(f"y_laborA = {-model.dual[model.laborA]}")
print(f"y_laborB = {-model.dual[model.laborB]}")

for c in [model.demand, model.laborA, model.laborB]:
    print(c.uslack())


Solution
x = 20.0
y = 60.0

Sensitivity Analysis
y_demand = 0.0
y_laborA = -20.0
y_laborB = -10.0
20.0
0.0
0.0


In [86]:
model.dual.pprint()

dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key      : Value
    dmd[Ams] :   1.0
    dmd[Ber] :   2.7
    dmd[Hag] :   0.8
    dmd[Lon] :   2.5
    dmd[Maa] :   1.8
    dmd[Utr] :   1.0
    src[Arn] :  -0.2
    src[Gou] :   0.0


In [77]:
model = ConcreteModel()
# for access to dual solution for constraints
model.dual = Suffix(direction=Suffix.IMPORT)

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
# model.profit = Objective(
#     expr = 40*model.x + 30*model.y,
#     sense = maximize)

@model.Objective(sense = maximize)
def profit(m):
    return 3*model.x + 5*model.y

@model.Constraint()
def fab_1(m):
    return model.x <= 4

@model.Constraint()
def fab_2(m):
    return 2*model.y <= 12

@model.Constraint()
def fab_3(m):
    return 3*model.x + 2*model.y <= 18

# solve
results = SolverFactory('cbc').solve(model)
results.write()

print("\nSolution")
print(f"x = {model.x()}")
print(f"y = {model.y()}")

str = "   {0:7.2f} {1:7.2f} {2:7.2f} {3:7.2f}"

print("Constraint  value  lslack uslack    dual")
for c in [model.fab_1, model.fab_2, model.fab_3]:
    print(c, str.format(c(), c.lslack(), c.uslack(), model.dual[c]))


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 36.0
  Upper bound: 36.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 2
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created sub

In [71]:
model.profit()

36.0

In [88]:
from pymprog import *

In [ ]:
# Creates the prob variable to contain the problem data
prob = LpProblem("Mix de Producao Fabricas", LpMinimize)

# Definicao das variaveis de decisao do problema
vars = LpVariable.dicts("Fab", F, 0, None, LpInteger)

# Definicao da funcao objetivo - minimizacao do custo
prob += (
    lpSum([vars[f] * Custo[f] for f in vars]),
    "Custo Producao",
)

# Definicao das restricoes
prob += (
    lpSum([vars[f] * MB[f] for f in vars]) <= 3_300,
    "Homem-Horas Disponiveis",
)

prob += (
    lpSum([vars[f] for f in vars]) == 1_000,
    "Minimo de veiculos produzido",
)

prob += (
    lpSum([vars[f] * MP[f] for f in vars]) <= 4000,
    "Materia prima disponivel",
)

prob += (
    vars['3'] >= 400,
    "Minimo_Prod_Fab_3" 
)


# The problem data is written to an .lp file
prob.writeLP("Mix_Producao_Automoveis.lp")

# The problem is solved using PULP's choice of solver 
prob.solve()

# The status of the solution is printed to the screen
print('Status: ', LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Total do Custo de Producao = ", value(prob.objective))